# MSG Data Wrangling Exercise
Paolo Rivas \
Email: privas.legua@gmail.com \
Phone #: 347 968-5109


## PART ONE
### Lender 1: Variable Transformation and Data Cleaning

In [1]:
#I will be doing almost most of my data cleaning using pandas library. Code is in Python 3.6.
import pandas as pd

#importint datasets skip the first rows with no info and and column headers
df1 = pd.read_excel(r'data/lender 1.xlsx', engine='openpyxl',skiprows=11, header=0)

#slicing data into the right info: 12 columns and 28 rows.
df2 = df1.iloc[0:28, 0:12]
df2.head(3)

,Year,School,City,State,School ID Number,Date of Financing,Loan,Type of Project,Percent of Students Proficient and Advanced in Math (School Level),Percent of Students Proficient and Advanced in Math (State Level),Percent of Students Proficient and Advanced in Reading (School Level),Percent of Students Proficient and Advanced in Reading (State Level)
0,2019.0,West Boulevard Charter,Woonsocket,RI,4.400029e+11,2019-01-31 00:00:00,6600000,Financing for renovations of a facility for an...,0.66,0.296,0.76,0.398
1,2019.0,West Boulevard Charter,Woonsocket,RI,4.400029e+11,2019-01-31 00:00:00,see above transaction,Financing for renovations of a facility for an...,0.66,0.296,0.76,0.398
2,2015.0,New Frontier Charter,Los Angeles,CA,6.227101e+10,2014-12-03 00:00:00,13022684,Financing for construction,0.36,0.4,0.4,0.51


In [2]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Year                                                                   28 non-null     float64
 1   School                                                                 28 non-null     object 
 2   City                                                                   28 non-null     object 
 3   State                                                                  28 non-null     object 
 4   School ID Number                                                       28 non-null     float64
 5   Date of Financing                                                      28 non-null     object 
 6   Loan                                                                   28 non-null     objec

### Solvind "see above transactions" problem
Three different strategies are possible fot this situation as far as I can tell. 
* The first one, called the ***lazy one and quick***, would imply transforming "see above transactions" into NA and getting read of all them in this date by using the pandas function df.dropna(inplace=True). However, because of the nature of this dataset and its lenght, this approach would do more harm than good. 
* The second one, called the ***hard and dumb***, would be individually transforming rows into the respective row  that we want (something like df2.loc[[2,3],:] = df1.loc[3].values). This would be valid if we only had to transform a truly small amount of values (no more than 3). Otherwise, that is extremely time consuming and not practical. From a coding perspective, a disaster. 
* The last one, which I call a ***middle ground***, consist on transforming the "see above transaction" string into a NA and then using a built-in python function called forward-fill that allows me to propagate the previous value forward once an NA is detected. **This approach is the one I am using.**


In [3]:
#replace see above transaction'values to NaN in the columns they exist. Then, replace NaN into last 
replace_df = df2[['Date of Financing', 'Loan', 'Type of Project']].replace('see above transaction', None).fillna(method='ffill')

#Updating values of columns with my new transformed variables
df2[['Date of Financing', 'Loan', 'Type of Project']] = replace_df

#bringing my new lenders 1 clean datset
df2.head(5)

,Year,School,City,State,School ID Number,Date of Financing,Loan,Type of Project,Percent of Students Proficient and Advanced in Math (School Level),Percent of Students Proficient and Advanced in Math (State Level),Percent of Students Proficient and Advanced in Reading (School Level),Percent of Students Proficient and Advanced in Reading (State Level)
0,2019.0,West Boulevard Charter,Woonsocket,RI,4.400029e+11,2019-01-31,6600000,Financing for renovations of a facility for an...,0.66,0.296,0.76,0.398
1,2019.0,West Boulevard Charter,Woonsocket,RI,4.400029e+11,2019-01-31,6600000,Financing for renovations of a facility for an...,0.66,0.296,0.76,0.398
2,2015.0,New Frontier Charter,Los Angeles,CA,6.227101e+10,2014-12-03,13022684,Financing for construction,0.36,0.4,0.4,0.51
3,2014.0,Accelerator MET School,Los Angeles,CA,6.227101e+10,2013-12-05,3200000,Financing for purchase,0.21,0.4,0.44,0.51
4,2013.0,Frank P. Porter School - High,Boston,MA,2.500071e+11,2012-11-30,3500000,Refinancing existing debt,0.76,0.49,0.74,0.52


In [4]:
"""Changing variables that are being detected as floats, objects or scientific 
notations into integer and correct data types"""

df2['Year'] = df2['Year'].astype(int)
df2['Loan'] = df2['Loan'].astype(int)
df2['School ID Number'] = df2['School ID Number'].astype(int)
df2['Date of Financing'] = df2['Date of Financing'].astype("datetime64[ns]")

### Last detail: get rid of duplicates (if they exist in our data set)
I have visually identified 2 duplicate in rows 2 (West Boulevard Charter) and 19 (New haven School, CA). Just to make sure there is no more duplicates in my dataset I will use another pandas built-in function to identify and get rid of repeating rows.

In [5]:
#Confirming the # of duplicates and non-duplicates
duplicates = df2.duplicated().sum()
nonduplicates = (~df2.duplicated()).sum()
print("Cases of duplicated values:", duplicates) 
print("Cases of non-duplicates values:", nonduplicates)

Cases of duplicated values: 2
Cases of non-duplicates values: 26


In [6]:
#Getting rid of the information that repeats in our dataset with drop_duplicates function.
final_lender1 = df2.drop_duplicates()

### Making sure data has been cleaned: 
As checked in the following information summary, we no longer have the 2 duplicate rows. Data is ready to export into a csv format.

In [7]:
#check my variables counts and data types
final_lender1.info()
#if needed you can download this new dataset by unmarking the following comment
#final_lender1.to_csv("Lender 1.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                                                                 Non-Null Count  Dtype         
---  ------                                                                 --------------  -----         
 0   Year                                                                   26 non-null     int64         
 1   School                                                                 26 non-null     object        
 2   City                                                                   26 non-null     object        
 3   State                                                                  26 non-null     object        
 4   School ID Number                                                       26 non-null     int64         
 5   Date of Financing                                                      26 non-null     datetime64[ns]
 6   Loan                                

## PART TWO
### Lender 2: Variable Transformation and Data Cleaning

In [8]:
#importint datasets skip the first rows with no info and and column headers
df3 = pd.read_excel(r'data/lender 2.xlsx', engine='openpyxl',skiprows=12, header=0)

#slicing data into the right info: 16 columns and 73 rows.
df4 = df3.iloc[0:73, 0:16]
df4.head(3)

/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Year,School,City,State,School ID Number,Date of Financing,Loan,Type of project,"Was financing for an addition, an existing building, or a new building?",Did the financing increase the amount of physical space available to educate students?,How many additional students can be accommodated?,How was the school served,Percent of Students Proficient and Advanced in Math (School Level),Percent of Students Proficient and Advanced in Math (State Level),Percent of Students Proficient and Advanced in Reading (School Level),Percent of Students Proficient and Advanced in Reading (State Level)
0,2007.0,Sheri Learning Academy ...,Indianapolis,IN,1.800058e+11,2007-07-20 00:00:00,3505000,Financing for renovating a facility owned by t...,Existing Building,Yes,300,School served by facilitating lending,NaN,NaN,NaN,NaN
1,2007.0,Indianapolis DREAM Middle School,Indianapolis,IN,1.800053e+11,2007-05-09 00:00:00,3600000,Financing for purchase,Existing Building,Yes,400,School served by facilitating lending,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,2007-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#as we can verify in the following info summary, there is a signficant ammount of levels of missing information.
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 16 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   Year                                                                                     40 non-null     float64
 1   School                                                                                   41 non-null     object 
 2   City                                                                                     40 non-null     object 
 3   State                                                                                    40 non-null     object 
 4   School ID Number                                                                         40 non-null     float64
 5   Date of Financing                                                 

### Solving the empty school names, empty dates, and empty loan values problems
Compared to the first lender dataset, in Lender 2 we have multiple problems that need to be solved with some assumptions. The lack of details on how the this information was collected is a major factor on the following decisions I will be making.
* **Assumption 1:** rows that **do not** contain names in the variable "School" are irrecuperable information. Therefore, will not be part of my data analysis.
* **Assumption 2:** rows that **have no information** in Date of Financing and Loan will also be excluded of the analysis due to their inoperability for the summary tables. 
* **Asumption 3:** Only those rows that **explicitly specify (SAME AS ABOVE)** will be considered repeated information from the previous row. 

In [10]:
#Subsetting only those rows that do not have an empty value in school
df5 = df4[df4['School'].notna()]

#subsetting only those rows that do not have an empty values at date of financing
df6 = df5[df4['Date of Financing'].notna()]

#subsetting only those rows that do not have empty values at Loan
df7 = df6[df4['Loan'].notna()]

/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [11]:
"""
After subsetting School, Date of Financing and Loan with only rows that have no missing information.
This is the shape of our lenders 2 dataset. From 73 entries, we now got 37.
"""
old_dim = df4.shape
new_dim = df7.shape
print("Old data shape:", old_dim)
print("New data shape after subsettings:", new_dim)

Old data shape: (73, 16)
New data shape after subsettings: (37, 16)


### Dealing with the (SAME AS ABOVE) issue
Taking into account this is a problem only presented in a specific area of our dataset, I will be executing the following strategy: 

1. Locking down my dataset into the specific rows and columns.
2. Transforming "Same as Above" into NaN values.
3. Filling NaNs with previous value.
4. Replacing original dataset with new transformations

In [12]:
#locking down my dataset into the specific rows and columns.
locked_df = df7.iloc[6:8, 5:12]
locked_df

,Date of Financing,Loan,Type of project,"Was financing for an addition, an existing building, or a new building?",Did the financing increase the amount of physical space available to educate students?,How many additional students can be accommodated?,How was the school served
9,2007-10-11 00:00:00,11450000,Financing for purchase,Existing Building,Yes,500,School served by facilitating lending
10,(SAME AS ABOVE),(SAME AS ABOVE),(SAME AS ABOVE),(SAME AS ABOVE),(SAME AS ABOVE),(SAME AS ABOVE),(SAME AS ABOVE)


In [13]:
#Transforming "Same as Above" into NaN values and Filling NaNs with previous value.
replaced_locked_df = locked_df.replace('(SAME AS ABOVE)', None).fillna(method='ffill')
replaced_locked_df

,Date of Financing,Loan,Type of project,"Was financing for an addition, an existing building, or a new building?",Did the financing increase the amount of physical space available to educate students?,How many additional students can be accommodated?,How was the school served
9,2007-10-11,11450000,Financing for purchase,Existing Building,Yes,500,School served by facilitating lending
10,2007-10-11,11450000,Financing for purchase,Existing Building,Yes,500,School served by facilitating lending


In [14]:
#Replacing original dataset with new transformations
df7.iloc[6:8, 5:12] = replaced_locked_df

/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [15]:
#Adding some extra transformation for incorrect data types
df7['Year'] = df7['Year'].astype(int)
df7['Loan'] = df7['Loan'].astype(int)
df7['School ID Number'] = df7['School ID Number'].astype(int)
df7['Date of Financing'] = df7['Date of Financing'].astype("datetime64[ns]")

/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set o

In [16]:
df7.head(3)

,Year,School,City,State,School ID Number,Date of Financing,Loan,Type of project,"Was financing for an addition, an existing building, or a new building?",Did the financing increase the amount of physical space available to educate students?,How many additional students can be accommodated?,How was the school served,Percent of Students Proficient and Advanced in Math (School Level),Percent of Students Proficient and Advanced in Math (State Level),Percent of Students Proficient and Advanced in Reading (School Level),Percent of Students Proficient and Advanced in Reading (State Level)
0,2007,Sheri Learning Academy ...,Indianapolis,IN,180005802312,2007-07-20,3505000,Financing for renovating a facility owned by t...,Existing Building,Yes,300,School served by facilitating lending,NaN,NaN,NaN,NaN
1,2007,Indianapolis DREAM Middle School,Indianapolis,IN,180005302408,2007-05-09,3600000,Financing for purchase,Existing Building,Yes,400,School served by facilitating lending,NaN,NaN,NaN,NaN
3,2007,Potomac Academy ...,Westminster,CO,80002006322,2007-05-31,2342842,Financing for a leasehold improvement,Existing Building,Yes,180,School served by facilitating lending,NaN,NaN,NaN,NaN


### Making sure data has been cleaned: 
As checked in the following information summary, We have no duplicates in Lenders 2 dataset.

In [17]:
#Confirming the # of duplicates and non-duplicates. No needp to drop_duplicate()
duplicates = df7.duplicated().sum()
nonduplicates = (~df7.duplicated()).sum()
print("Cases of duplicated values:", duplicates) 
print("Cases of non-duplicates values:", nonduplicates)

Cases of duplicated values: 0
Cases of non-duplicates values: 37


In [18]:
final_lender2 = df7.drop_duplicates()

In [19]:
#check my variables counts and data types
final_lender2.info()
#if needed you can download this new dataset by unmarking the following comment
#final_lender1.to_csv("Lender 2.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 70
Data columns (total 16 columns):
 #   Column                                                                                   Non-Null Count  Dtype         
---  ------                                                                                   --------------  -----         
 0   Year                                                                                     37 non-null     int64         
 1   School                                                                                   37 non-null     object        
 2   City                                                                                     37 non-null     object        
 3   State                                                                                    37 non-null     object        
 4   School ID Number                                                                         37 non-null     int64         
 5   Date of Financing

## PART THREE
### Lender 1 & 2: Joining datasets and create summary tables
Some considerations before joining our datasets.
1. A considerable ammount of information is irrelevant for creating the required summary table.
2. Reducing dimensions and complexity will also reduce the ammount of missing values.
3. Concentrating in matching ID (school number ID) and required independent variables (school name, year and Loan values) would be enough to satisfy all summary tables. Therefore, no need to use extra variables.

In [20]:
#Createing a reduced version of our datasets

L1_df = final_lender1[['Year', 'School', 'Date of Financing','School ID Number', 'Loan']]
L2_df = final_lender2[['Year', 'School', 'Date of Financing','School ID Number', 'Loan']]

print(L1_df.columns)
print(L2_df.columns)

Index(['Year', 'School', 'Date of Financing', 'School ID Number', 'Loan'], dtype='object')
Index(['Year', 'School', 'Date of Financing', 'School ID Number', 'Loan'], dtype='object')


In [21]:
L2_df.shape

(37, 5)

#### Checking how many IDs we have in common between datasets

In [22]:
pd.merge(L1_df, L2_df, on=['School ID Number'], how='inner')

,Year_x,School_x,Date of Financing_x,School ID Number,Loan_x,Year_y,School_y,Date of Financing_y,Loan_y
0,2007,Indianapolis DREAM School,2007-03-30,180005302408,3600000,2007,Indianapolis DREAM Middle School,2007-05-09,3600000


### Interesting finding and new assumptions
* The only interception within our datasets is at the Indianapolis DREAM School. Interestingly, the interception is also revealing of another relevant pattern: some schools might have the same ID but a different variation of school name or the same school name but a different levels (high, middle, elementary) and school ID. To sort this out, some new assumptions have to be made.
* **Assumption 4:** Only schools who share the same Name regardless of the level (High, Middle, Elementary) or location (New York, Pittsburgh) will be considered the same school. In contrast, having the same School ID does not qualify a school as the same school unit, unless it also shares the same name.


In [23]:
#Renaming Schools into single units

#All Frank P. Porter School regardless of levels (high, elementary) are the same school.
L1_df.School = L1_df.School.str.replace('Frank P. Porter School - High', "Frank P. Porter School")
L1_df.School = L1_df.School.str.replace('Frank P. Porter School - Elementary', "Frank P. Porter School")

/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
#All DREAM schools regardless of location are the same school
L1_df.School = L1_df.School.str.replace('DREAM Charter School', "DREAM School")
L1_df.School = L1_df.School.str.replace('West DREAM School', "DREAM School")
L1_df.School = L1_df.School.str.replace('New York DREAM School', "DREAM School")
L1_df.School = L1_df.School.str.replace('Indianapolis DREAM School', "DREAM School")
L2_df.School = L2_df.School.str.replace('Indianapolis DREAM Middle School', "DREAM School")

In [25]:
#All PA schools regardless of locations are the same school
L1_df.School = L1_df.School.str.replace('PA Prep Pittsburgh', "PA Prep")

In [26]:
#check replace in L1_df
L1_df.School

0                  West Boulevard Charter
2                    New Frontier Charter
3                  Accelerator MET School
4                 Frank P. Porter School 
5                  Frank P. Porter School
6                      New School Charter
7                           LA New School
8                       Event City School
9                            Oakland Prep
10                   Desert Middle School
11                      Holt Humbolt High
12                           DREAM School
13                             LA Academy
14                   Sunny Day Montessori
15           Lightning in a Bottle School
16                 New Beginnings Academy
17                       New Haven School
19             School of Science and Arts
20                        Oakland Academy
21                           DREAM School
22          Jackson Jordan Charter School
23    School of Science and Technology LA
24                           DREAM School
25                           DREAM

In [27]:
#check replace in L2_df
L2_df.School

0     Sheri Learning Academy                        ...
1                                          DREAM School
3     Potomac Academy                               ...
5     Star School                                   ...
6     Roses High School                             ...
7              Oregon Prep                             
9     Potomac Prep                                  ...
10           Washington School                         
13                     Presidential Leadership Academy 
16    Potomac Prep                                  ...
17           Washington School                         
18                                 Boise Charter School
20                                          Cap Academy
21                                        Junction Prep
22                                       CO School Prep
24                                      Phoenix Academy
25                                      Phoenix Academy
27                                Porter Charter

#### Relevant observation: noticing some white spaces that will be problematic later so I will deal with it now.

In [28]:
L1_df.School = L1_df.School.str.strip()
L1_df.School

0                  West Boulevard Charter
2                    New Frontier Charter
3                  Accelerator MET School
4                  Frank P. Porter School
5                  Frank P. Porter School
6                      New School Charter
7                           LA New School
8                       Event City School
9                            Oakland Prep
10                   Desert Middle School
11                      Holt Humbolt High
12                           DREAM School
13                             LA Academy
14                   Sunny Day Montessori
15           Lightning in a Bottle School
16                 New Beginnings Academy
17                       New Haven School
19             School of Science and Arts
20                        Oakland Academy
21                           DREAM School
22          Jackson Jordan Charter School
23    School of Science and Technology LA
24                           DREAM School
25                           DREAM

In [29]:
L2_df.School = L2_df.School.str.strip()
L2_df.School

0              Sheri Learning Academy
1                        DREAM School
3                     Potomac Academy
5                         Star School
6                   Roses High School
7                         Oregon Prep
9                        Potomac Prep
10                  Washington School
13    Presidential Leadership Academy
16                       Potomac Prep
17                  Washington School
18               Boise Charter School
20                        Cap Academy
21                      Junction Prep
22                     CO School Prep
24                    Phoenix Academy
25                    Phoenix Academy
27              Porter Charter School
28          Two Rivers Charter School
31                 Blue Ridge Academy
34         Spirit Empowerment Academy
37                        Oregon Prep
38                      NM New School
40                    Phoenix Academy
43                     CO School Prep
45            Bear Leadership Academy
48          

### Mergin datasets with new lenders variable
I will create a new variable "Lender" before merging our dataframes with pandas concat() function (no need for any kind of joining in this case)

In [30]:
#creating new variable lender in both DFs
L1_df['Lender'] = 'Lender1'
L2_df['Lender'] = 'Lender2'

/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/paolorivas/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
Lenders_complete_df = pd.concat([L1_df, L2_df])

In [32]:
Lenders_complete_df
#Lenders_complete_df.to_csv("Data_lenders.csv")

,Year,School,Date of Financing,School ID Number,Loan,Lender
0,2019,West Boulevard Charter,2019-01-31,440002900495,6600000,Lender1
2,2015,New Frontier Charter,2014-12-03,62271012511,13022684,Lender1
3,2014,Accelerator MET School,2013-12-05,62271013274,3200000,Lender1
4,2013,Frank P. Porter School,2012-11-30,250007101748,3500000,Lender1
5,2013,Frank P. Porter School,2012-11-30,250052802712,3500000,Lender1
...,...,...,...,...,...,...
63,2015,Wilmington School for the Arts,2014-12-09,100005400361,350000,Lender2
65,2016,SPORTs Academy,2016-04-20,480009208162,699629,Lender2
67,2018,Arts Academy Phoenix,2018-05-15,40093803479,250000,Lender2
68,2019,Three Rivers School,2018-10-24,40024803081,5525000,Lender2


#### Self criticism to the approach taken: 
According to the given description "Some transactions include multiple schools". However, my approach is asumming that any transaction that does not match school name, date of financing and School ID is not the same transaction, even though the loan number is the same. It is my understanding that it could be the case that a same amount could be given to a same institution in different moments. For that reason,and the lack of more information, I decided to proceed like previously explained. However, this could bring some limitations to the accuracy of the final summary table numbers.


In [33]:
#Just in case, to solidify my argument I will be checking duplicates. No duplicates is good news for my approach.
duplicates = Lenders_complete_df.duplicated().sum()
nonduplicates = (~Lenders_complete_df.duplicated()).sum()
print("Cases of duplicated values:", duplicates) 
print("Cases of non-duplicates values:", nonduplicates)

Cases of duplicated values: 0
Cases of non-duplicates values: 63


### Creating Summary tables!

**1. The sum of loans amounts by lender by year without double counting.**

In [34]:
table1 = Lenders_complete_df.groupby(by=['Year','Lender']).Loan.sum() 
pd.DataFrame(table1)

Loan
Year Lender           
2007 Lender1  18608293
     Lender2   9447842
2008 Lender1  46529021
     Lender2  31882519
2009 Lender1  51503780
     Lender2   8673203
2010 Lender1   9285715
     Lender2   3987886
2011 Lender2  12104805
2012 Lender1  44172334
     Lender2   1021000
2013 Lender1   7000000
     Lender2  12933963
2014 Lender1   3200000
     Lender2   6392000
2015 Lender1  13022684
     Lender2   1110500
2016 Lender2    699629
2018 Lender2    250000
2019 Lender1   6600000
     Lender2  12685000

**2.	The number of schools served by lender by year.**

In [35]:
# By using nunique() I am making sure only a unique value is being selected. Ergo, one school per query.
table2 = Lenders_complete_df.groupby(by=['Year','Lender']).School.nunique()
pd.DataFrame(table2)

School
Year Lender         
2007 Lender1       2
     Lender2       3
2008 Lender1       7
     Lender2       5
2009 Lender1       6
     Lender2       7
2010 Lender1       1
     Lender2       1
2011 Lender2       6
2012 Lender1       3
     Lender2       2
2013 Lender1       1
     Lender2       4
2014 Lender1       1
     Lender2       2
2015 Lender1       1
     Lender2       3
2016 Lender2       1
2018 Lender2       1
2019 Lender1       1
     Lender2       2

**3. The total amount of loans serviced by each lender across all years without duplicate information.**

In [36]:
table3 = Lenders_complete_df.groupby(by=['Lender']).Loan.count()
pd.DataFrame(table3)

,Loan
Lender,
Lender1,26
Lender2,37


**4. The total number schools served across all years by lender without double counting schools.**

In [37]:
# By using nunique() I am making sure only a unique value is being selected. Ergo, one school per query.
table4 = Lenders_complete_df.groupby(by=['Lender']).School.nunique()
pd.DataFrame(table4)

,School
Lender,
Lender1,21
Lender2,29


**5. The total amount of loans serviced by both lenders across all years without duplicate information.** 

In [38]:
table5 = Lenders_complete_df.Loan.count()
print("total ammount",table5)

total ammount 63


**6. The total number schools served by both lenders across all years without double counting schools.** 

In [39]:
# By using nunique() I am making sure only a unique value is being selected. Ergo, one school per query.
table6 = Lenders_complete_df.School.nunique()
print("total ammount",table6)

total ammount 49
